<div style="padding-bottom:30px">
<a href="https://github.com/cwbeitel/inquiry"><img src="https://raw.githubusercontent.com/cwbeitel/iqassets/master/logotype_blue_small.png" style="width:100px; margin-left:0px"></img></a>
<p style="color:#9E9E9E">
<a href="https://github.com/cwbeitel/inquiry/tree/master/docs">Getting Started Guide</a> // <a href="https://goo.gl/forms/2cOmuUrQ3n3CKpim1">Documentation Feedback</a></p>
</div>

<h1 style="color:#9E9E9E">MS preprocessing and MDV analysis with XCMS3</h1>

This notebook demonstrates an LCMS data preprocessing and MDV analysis workflow with XCMS3. Read more about [metabolomics](https://en.wikipedia.org/wiki/Metabolomics) or the XCMS [project](https://xcmsonline.scripps.edu/landing_page.php?pgcontent=mainPage) or [toolkit](https://github.com/sneumann/xcms).

In [1]:
library(xcms)
library(RColorBrewer)
register(SerialParam())
library("IRdisplay")
rm(list = setdiff(ls(), lsf.str()))

Loading required package: Biobase
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, cbind, colMeans, colnames,
    colSums, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, lengths, Map, mapply, match,
    mget, order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rowMeans, rownames, rowSums, sapply, setdiff, sort,
    table, tapply, union, unique, unsplit, which, which.max, which.min

Welcome to Bioconductor

    Vignettes contain intro

In [2]:
load_pheno <- function(files) {
    s_groups = seq(from=1, to=length(files))
    # HACK: For now doing this but todo move to reading from file if we want to be able to
    # color by subset but then again it is useful to color each sample differently when
    # performing initial QC.
    pheno <- data.frame(sample_name = sub(basename(files), pattern = ".mzML",
                                          replacement = "", fixed = TRUE),
                        sample_group = s_groups, stringsAsFactors = FALSE)
    return(pheno)
}

prepare_arguments <- function(isotopolog_def_path, inputs_dir, output_dir, compound_tags) {
    files = list.files(inputs_dir,  recursive=TRUE, pattern='mzML', full.name=TRUE)
    pheno <- load_pheno(files)
    params <- list("files" = files,
                   "pheno" = pheno,
                   "isotopolog_def_path" = isotopolog_def_path,
                   "output_dir" = output_dir,
                   "show_figures_inline" = TRUE,
                   "compound_tags"=compound_tags)
    return(params)
}

compound_tags = c("Gly", "Thr", "Leu", "Gln", "Lys", "Met", "His", "Arg")
params = prepare_arguments('/Users/cb/workspace/infra/inquiry/inquiry/toolkit/metabolomics/test/compounds.txt', '/tmp/iq/converted', '/tmp/iq/msout', compound_tags)
raw_data = readMSData2(params$files, pdata = new("NAnnotatedDataFrame", params$pheno))

In [3]:
sample_colors_for_dataset <- function(raw_data) {
    labels <- unique(pData(raw_data)$sample_group)
    sample_colors <- palette(rainbow(length(labels)))
    names(sample_colors) <- labels
    return(sample_colors)
}

plot_tic_spectrum <- function(raw_data, params, files=c(1), tag="untitled", all=FALSE) {
    print("Visualizing TIC spectrum...")
    sample_colors <- sample_colors_for_dataset(raw_data)
    
    fig_out_path = paste(params$output_dir, '/', tag, '-initial_tic_spectrum.png', sep='')
    png(fig_out_path, width=8.5, height=5.5, units="in", res=300, pointsize=4)

    if(all){
        files = raw_data$sample_group
    }
    
    tmp <- filterFile(raw_data, file = files[1])
    plot(x = rtime(tmp), y = tic(tmp), xlab = "retention time", ylab = "TIC",
         col = sample_colors[pData(tmp)$sample_group],
         type = "l")
    legend("topleft", col = sample_colors, legend = names(sample_colors), lty = 1)
    
    if(length(files)>1){
        for(i in seq(2,length(files))){
            tmp <- filterFile(raw_data, file=files[i])
            lines(x = rtime(tmp), y = tic(tmp), xlab = "retention time", ylab = "TIC",
                 col = sample_colors[pData(tmp)$sample_group], type = "l")        
        }
    }
    
    dev.off()
        
    print("Done.")
}

plot_tic_boxplot <- function(raw_data, params) {
    print("Visualizing TIC boxplot...")
    sample_colors <- sample_colors_for_dataset(raw_data)
    
    fig_out_path = paste(params$output_dir, '/initial_tic_boxplot.png', sep='')
    png(fig_out_path, width=8.5, height=5.5, units="in", res=300, pointsize=4)
    
    tc <- split(tic(raw_data), f = fromFile(raw_data))
    boxplot(tc,
            col = sample_colors[pData(raw_data)$sample_group],
            ylab = "intensity", main = "Total ion current")
    
    dev.off()

    print("Done.")
}

plot_bpc <- function(raw_data, params) {
    print("Visualizing BPC...")
    sample_colors <- sample_colors_for_dataset(raw_data)
    
    fig_out_path = paste(params$output_dir, '/initial_bpc_spectrum.png', sep='')
    png(fig_out_path, width=8.5, height=5.5, units="in", res=300, pointsize=4)

    ## Get the base peak chromatograms. This reads data from the files.
    bpis <- extractChromatograms(raw_data, aggregationFun = "max")
    plot(3, 3, pch = NA, xlim = range(unlist(lapply(bpis, rtime))),
         ylim = range(unlist(lapply(bpis, intensity))), main = "BPC",
         xlab = "rtime", ylab = "intensity")
    for (i in 1:length(bpis)) {
        points(rtime(bpis[[i]]), intensity(bpis[[i]]), type = "l",
           col = sample_colors[pData(raw_data)$sample_group[i]])
    }
    
    dev.off()

    print("Done.")
}

In [4]:
plot_tic_spectrum(raw_data, params, files=c(i), tag='all', all=TRUE)

[1] "Visualizing TIC spectrum..."


ERROR: Error in names(sample_colors) <- labels: 'names' attribute [16] must be the same length as the vector [8]


In [ ]:
for(i in seq(1:length(unique(pData(raw_data)$sample_group)))){
    plot_tic_spectrum(raw_data, params, files=c(i), tag=i)
}

plot_tic_spectrum(raw_data, params, files=c(i), tag='all', all=TRUE)
plot_tic_boxplot(raw_data, params)
#plot_bpc(raw_data, params)

In [ ]:
# Explore 
keep_samples <- c(1,2,3,8,9,10)
plot_tic_spectrum(raw_data, params, files=keep_samples, tag='keep')
retained = filterFile(raw_data, file=keep_samples)

In [ ]:
identify_initial_peaks <- function(raw_data, params) {
    print("== (Step 2/6) Running initial peak identification and summary analysis. ==")
    sample_colors <- sample_colors_for_dataset(raw_data)
    cwp <- CentWaveParam(snthresh = 20, noise = 1000)
    xod <- findChromPeaks(raw_data, param = cwp)
    print("== (Step 2/6) Done. ==")
    return(xod)
}

viz_chrom_peaks <- function(xod, params) {
    sample_colors <- sample_colors_for_dataset(xod)
    
    ints <- split(chromPeaks(xod)[, "into"], f = chromPeaks(xod)[, "sample"])
    ints <- lapply(ints, log2)
    png(paste(params$output_dir, '/initial_peak_intensities.png', sep=''), width=8.5, height=5.5, units="in", res=300, pointsize=4)
    boxplot(ints, varwidth = TRUE, col = sample_colors[pData(xod)$sample_group],
        ylab = expression(log[2]~intensity), main = "Peak intensities")
    dev.off()
}

xod = identify_initial_peaks(retained, params)
viz_chrom_peaks(xod, params)


In [ ]:
# adjust_and_analyze_rt <- function(xod, params) {
#     print("== (Step 3/6) Performing initial RT adjustment and analysis. ==")
#     xod <- adjustRtime(xod, param = ObiwarpParam())
    
#     #plot_bpc(xod)
    
#     ## Plot also the difference of adjusted to raw retention time.
#     plotAdjustedRtime(xod, col=sample_colors[pData(xod)$sample_group])

#     ## Calculate the difference between the adjusted and the raw retention times.
#     diffRt <- rtime(xod) - rtime(xod, adjusted = FALSE)

#     ## By default, rtime and most other accessor methods return a numeric vector. To
#     ## get the values grouped by sample we have to split this vector by file/sample
#     diffRt <- split(diffRt, fromFile(xod))

#     png(params$output_dir + '/primary_adjust_rt.png', width=8.5, height=5.5, units="in", res=300, pointsize=4)
#     boxplot(diffRt,
#             col = sample_colors[pData(xod)$sample_group],
#             main = "Obiwarp alignment results",
#             ylab = "adjusted - raw rt")
#     print("== (Step 3/6) Done. ==")
#     dev.off()
    
#     return(xod)
# }

# xod = adjust_and_analyze_rt(xod, params)

In [ ]:
# secondary_group_peaks <- function(xod) {
#     print("== (Step 4/6) Performing secondary peak grouping. ==")
#     ## Define the PeakDensityParam
#     pdp <- PeakDensityParam(sampleGroups = pData(xod)$sample_group,
#                 maxFeatures = 300, minFraction = 0.66)
#     xod <- groupChromPeaks(xod, param = pdp)    
    
#     # The set of peak "features" used in groupChromPeaks can be displayed as follows
#     print('Displaying feature definitions:')
#     print(head(featureDefinitions(xod)))
    
#     # The following allows us to see where data is missing and will be filled
#     print('Displaying sample of featureValues() before infilling...')
#     print(head(featureValues(xod, value = "into")))
    
#     # Performing missing data infilling from original unfiltered data files
#     xod <- fillChromPeaks(xod)

#     # Displaying the infilling
#     head(featureValues(xod, value = "into"))

#     # Write featureDefinitions(xod) to disk.
#     # TODO: Write processed feature values to disk.

#     print("== (Step 4/6) Done. ==")
# }

# xod = secondary_group_peaks(xod)

In [ ]:
# plot_adjusted_rtime <- function(xod, params) {
#     sample_colors <- sample_colors_for_dataset(xod)
#     plotAdjustedRtime(xod, col = sample_colors[pData(xod)$sample_group])
# }

# perform_secondary_rt_adjustment <- function(xod, params) {
#     print("== (Step 5/6) Performing secondary retention time adjustment. ==")
#     ## Define the parameter for the correspondence
#     pdparam <- PeakDensityParam(sampleGroups = pData(xod)$sample_group,
#                     minFraction = 0.7, maxFeatures = 100)
#     xod <- groupChromPeaks(xod, param = pdparam)

#     ## Create the parameter class for the alignment
#     pgparam <- PeakGroupsParam(minFraction = 0.9, span = 0.4)

#     ## Extract the matrix with (raw) retention times for the peak groups that would
#     ## be used for alignment.
#     adjustRtimePeakGroups(xod, param = pgparam)
    
#     # Adjust rtime based on peak groups
#     xod <- adjustRtime(xod, param = pgparam)
    
#     png(params$output_dir + '/secondary_adjust_rt.png')
#     plot_adjusted_rtime(xod)
#     dev.off()
    
#     print("== (Step 5/6) Done. ==")
#     return(xod)
# }

# xod = perform_secondary_rt_adjustment(xod, params)

In [ ]:
ms_neighborhood <- function(d, rt, mz, mz_window=0.5, rt_window=1){
    print(paste("Extracting ms neighborhood around rt", str(rt), "and mz", str(mz)))
    rt_frac <- 0.07
    rtr <- c(rt - rt_window/2, rt + rt_window/2)
    mzr <- c(mz - mz_window/2, mz + mz_window/2)
    return(extractMsData(d, mz=mzr, rt=rtr))
}

plot_slice <- function(d, rt, mzr, mz_frac, sample=1, tag="Untagged", params){
    print(paste("Visualizing m/z x itensity slice for rt", rt, "and mz range", mzr))
    rt_frac <- 0.05
    mzr <- c(mzr[1]*(1-mz_frac), mzr[2]*(1+mz_frac))
    rtr <- c(rt*(1-rt_frac), rt*(1+rt_frac))
    d <- extractMsData(d, mz=mzr, rt=rtr)[[1]]
    png(paste(params$output_dir, '/', tag, '-', mz_frac, '-', rt, '-', 'slice.png', sep=''), width=8.5, height=5.5, units="in", res=300, pointsize=4)
    plot(d$mz, d$i, type="l", ylab="intensity", xlab="m/z")
    title(paste(tag, "; ", "rt=", rt, "+/-", paste(rt_frac*100, "%", sep="")))
    dev.off()
}

In [ ]:
load_isotopolog_defs <- function(path) {
    print('Loading isotopolog definitions table...')
    t = read.table(path, sep=' ', header=1)
    t$RT <- t$RT*60 # Minutes to seconds.
    print('Done. A preview of the defs table:')
    print(head(t))
    return(t)
}

In [ ]:
t = load_isotopolog_defs(params$isotopolog_def_path)
t

In [ ]:
generate_mdv_figures <- function(xod, rt, mzr, tag, mzvals, sample, params, make_spectra=FALSE) {
    if(make_spectra){
        for(scale in c(1, 0.05, 0.0001)) {
            outpath = paste(params$output_dir, '/mdv_spectrum_s', scale, '_t', tag, '.png', sep='')
            png(outpath)
            plot_slice(xod, rt, mzr, scale, sample=1, tag=tag, params=params)
            dev.off()
        }
    }
    png(paste(params$output_dir, '/', tag, '-mdv_barplot.png', sep=''))
        barplot(mzvals/sum(mzvals), xlab="Isotopolog", ylab="Fraction")
        title(paste("Isotopolog fractions, ", tag))
    dev.off()
}

analyze_sample_mdvs <- function(xod, t, tag, sample, params, make_spectra=FALSE) {
    hits = grep(tag, t$Name)
    rt <- t[hits[1],]$RT
    mz <- t[hits,]$Mass
    mzr <- c(min(mz), max(mz))
    mzvals <- c()
    for(mzv in mz){mzvals <- c(mzvals, max(ms_neighborhood(xod, rt, mzv)[[sample]]$i))}=
    generate_mdv_figures(xod, rt, mzr, tag, mzvals, sample, params, make_spectra)
    mdvs <- mzvals/sum(mzvals)
    mdv_out_path <- paste(params$output_dir, '/', tag, '-mdvs.txt', sep='')
    write.table(mdvs, mdv_out_path)
    return(mdvs)
}

mdvs = analyze_sample_mdvs(xod, t, tag="Arg", sample=1, params=params, make_spectra=TRUE)

In [ ]:
analyze_mdvs <- function(xod, t, tag, sample, params) {
    print("== (Step 6/6) Analyzing sample MDVs. ==")
    for(tag in params$compound_tags) {
        mzvals <- analyze_sample_mdvs(xod, t, tag, sample, params)
    }
    print("== (Step 6/6) Done.")
}

analyze_mdvs(xod, t, "mdvs", 1, params)

<h3 style="color:#9E9E9E">References</h3>

1. Michael Droettboom, Thomas A Caswell, John Hunter, Eric Firing, Jens Hedegaard Nielsen, Nelle Varoquaux, … Nikita Kniazev. (2017). matplotlib/matplotlib v2.0.1 [Data set]. Zenodo. http://doi.org/10.5281/zenodo.570311
2. Gatto L and Lilley K (2012). “MSnbase - an R/Bioconductor package for isobaric tagged mass spectrometry data visualization, processing and quantitation.” Bioinformatics, 28, pp. 288-289.
3. Smith, C.A., Want, E.J., O'Maille, G., Abagyan,R., Siuzdak and G. (2006). “XCMS: Processing mass spectrometry data for metabolite profiling using nonlinear peak alignment, matching and identification.” Analytical Chemistry, 78, pp. 779–787.
4. Tautenhahn R, Boettcher C and Neumann S (2008). “Highly sensitive feature detection for high resolution LC/MS.” BMC Bioinformatics, 9, pp. 504.
5. Benton HP, Want EJ and Ebbels TMD (2010). “Correction of mass calibration gaps in liquid chromatography-mass spectrometry metabolomics data.” BIOINFORMATICS, 26, pp. 2488.

<h3 style="color:#9E9E9E">Contact</h3>

Want to get in touch? You can [provide feedback](https://goo.gl/forms/2cOmuUrQ3n3CKpim1) regarding this or other documentation,
[reach out to us](https://goo.gl/forms/j8FWdNJqABAoJvcW2) regarding collaboration, or [request a new feature or analytical capability](https://goo.gl/forms/dQm3SDcoNZsV7AAd2). We're looking forward to hearing from you!

<div style="padding-top: 30px">
<p style="color:#9E9E9E; text-align:center">This notebook was prepared for <a href="https://github.com/cwbeitel/inquiry">Project Inquiry</a> in support of the research mission of the Joint BioEnergy Institute (JBEI). Learn more at https://www.jbei.org/.</p>
<p style="color:#9E9E9E; text-align:center">The Joint BioEnergy Institute is a program of the U.S. Department of Energy Office of Science.</p>
<p style="color:#9E9E9E; text-align:center">© Regents of the University of California, 2017. Licensed under a BSD-3 <a href="https://github.com/cwbeitel/inquiry/blob/master/LICENSE">license</a>.</p>
<img src="https://raw.githubusercontent.com/cwbeitel/iqassets/master/logotype_blue_small.png" style="width:100px"></img>
</div>